In [ ]:
from io import BytesIO
from os import write
import struct
import os

filename = "Z:\\인프런-주피터\\파이썬실용프로젝트10선\\MP3\\bensound-newdawn.mp3"
COVER_IMAGE = "Z:\\인프런-주피터\\파이썬실용프로젝트10선\\MP3\\girl_cover.jpg"

TIF = [
    ("TALB", "앨범제목"),
    ("TPE1", "아무개1"),
    ("TPE2", "아무개2"),
    ("TPE3", "아무개3"),
    ("TCON", "호러"),
    ("TCOM", "작곡가"),
    ("TIT1", "제목1"),
    ("TIT2", "제목2"),
    ("TCOP", "벤사운드"),
    ("TYER", "1800"),
    ("TRCK", "1/10"),
    ("TPOS", "1/2")
]

USLT = {
    "lang": "kor",
    "content_descript": "설명",
    "lyrics": "파이썬으로 만드는 MP3 태그\n노래가사 테스트\n남박사의 알쓸파잡!!!"
}

COMMENTS = {
    "lang": "kor",
    "content_descript": "",
    "comments": "코멘트입니다.\n코멘트도 개행문자가 포함될 수 있습니다."
}


def decode_size(str_hex):
    slice_hex = [str_hex[i:i+2] for i in range(0, len(str_hex), 2)]
    results = ""
    for h in slice_hex:
        results += "{0:b}".format(int(h, 16)).zfill(7)
    return int(results, 2)


def encode_size(realbytes):
    hv = "{:08x}".format(realbytes)
    size = [hv[i:i+2] for i in range(0, len(hv), 2)]
    tmp_result = ""
    for n in size:
        tmp_result += "{:08b}".format(int(n, 16))

    tmp_result = tmp_result[4:]
    hexs = ["0" + tmp_result[i:i+7] for i in range(0, len(tmp_result), 7)]
    sa = []
    for h in hexs:
        sa.append("{:02x}".format(int(h, 2)))
    return "".join(sa)


mp3 = open(filename, "rb")
header_size = 10
header_data = mp3.read(header_size)
id3_header = BytesIO(header_data)

ID3_TAG = id3_header.read(3)
ID3_VERSION = id3_header.read(2)
ID3_FLAGS = id3_header.read(1)
ID3_SIZE = id3_header.read(4)
ID3_BYTES = decode_size(ID3_SIZE.hex())
ID3_DATA = BytesIO(mp3.read(ID3_BYTES))

delete_tag_mp3 = BytesIO(mp3.read())
mp3.close()

def write_text(enc_text, bio):
    bio.write(b"\x01")
    bio.write(enc_text)
    bio.write(b"\x00")

# ID3 태그 전체를 기록할 BytesIO 변수
ID3_FRAMES = BytesIO()

# TIF 내용을 모두 기록합니다.
for t in TIF:
    frame = BytesIO()
    fid, value = t
    encoding = value.encode("UTF-16")
    frame.write(fid.encode())
    frame.write(struct.pack(">I", len(encoding) + 2))
    frame.write(struct.pack("2s", b"\x00"))
    write_text(encoding, frame)
    frame.seek(0)
    ID3_FRAMES.write(frame.read())

# 가사데이터를 기록합니다.
if USLT:
    lang = USLT.get("lang")
    content_descriptor = USLT.get("content_descript")
    lyrics = USLT.get("lyrics")
    encoding_desc = content_descriptor.encode("UTF-16")
    encoding_lyrics = lyrics.encode("UTF-16")

    # 실 데이터를 BytesIO() 에 담아 따로 관리 합니다.
    data = BytesIO()
    data.write(b"\x01")
    data.write(b"kor")
    data.write(encoding_desc)
    data.write(b"\x00")
    data.write(b"\x00")
    data.write(encoding_lyrics)
    data.write(b"\x00")
    data.seek(0)

    # 헤더 작성을 위해 실 데이터가 담긴 변수의 총 크기를 구합니다.
    frame_size = data.getbuffer().nbytes
    # 실 데이터와 프레임 헤더를 작성할 BytesIO 를 선언합니다.
    frame = BytesIO()
    frame.write(b"USLT")
    frame.write(struct.pack(">I", frame_size))
    frame.write(struct.pack("2s", b"\x00"))
    frame.write(data.read())
    frame.seek(0)
    # ID3_FRAMES 에 기록합니다.
    ID3_FRAMES.write(frame.read())

if COMMENTS:
    lang = COMMENTS.get("lang")
    content_descriptor = COMMENTS.get("content_descript")
    comments = COMMENTS.get("comments")
    encoding_desc = content_descriptor.encode("UTF-16")
    encoding_comments = comments.encode("UTF-16")
    
    data = BytesIO()
    data.write(b"\x01")
    data.write(b"kor")
    data.write(encoding_desc)
    data.write(b"\x00")
    data.write(b"\x00")
    data.write(encoding_comments)
    data.write(b"\x00")
    data.seek(0)

    frame_size = data.getbuffer().nbytes
    frame = BytesIO()
    frame.write(b"COMM")
    frame.write(struct.pack(">I", frame_size))
    frame.write(struct.pack("2s", b"\x00"))
    frame.write(data.read())
    frame.seek(0)
    ID3_FRAMES.write(frame.read())

# 커버 이미지 추가
if os.path.exists(COVER_IMAGE):
    mime_type = ""
    binary = None
    # 이미지 파일을 오픈합니다.
    with open(COVER_IMAGE, "rb") as f:
        # 이미지 파일의 전체를 읽습니다.
        binary = f.read()
        # 이미지 파일의 앞 3바이트로 이미지 형식을 판단합니다.
        # 파이썬에는 mime_type 을 추측할 수 있는 외부 라이브러리가 있으나
        # 여기서는 사용하지 않습니다.
        header = binary[0:3]
        if header == b"\xff\xd8\xff":
            mime_type = "image/jpeg"
        elif header == b"\x47\x49\x46":
            mime_type = "image/gif"
        elif header == b"\x89\x50\x4e":
            mime_type = "image/png"
    # 파일이 정상적이고 mime 형식이 존재하면 이미지를 추가합니다.
    if binary is not None and mime_type != "":
        # 이미지 실데이터를 작성합니다.
        img_frame = BytesIO()
        img_frame.write(b"\x01")
        img_frame.write(mime_type.encode())
        img_frame.write(b"\x00")
        img_frame.write(b"\x03") # 커버 전면 이미지로 설정
        img_frame.write("이미지 설명".encode("UTF-16"))
        img_frame.write(b"\x00")
        img_frame.write(b"\x00")
        img_frame.write(binary)
        img_frame.seek(0)

        # 이미지 실제 데이터의 크기를 구합니다.
        frame_size = img_frame.getbuffer().nbytes

        # 실 데이터와 헤더를 작성합니다.
        frame = BytesIO()
        frame.write(b"APIC")
        frame.write(struct.pack(">I", frame_size))
        frame.write(struct.pack("2s", b"\x00"))
        frame.write(img_frame.read())
        frame.seek(0)

        # 최종 ID3_FRAMES 에 작성합니다.
        ID3_FRAMES.write(frame.read())

# 최종 ID3_FRAMES 의 파일포인터를 맨 앞으로 옮깁니다.
ID3_FRAMES.seek(0)

# 최종 ID3_FRAMES 의 전체 크기를 구합니다.
frame_size = ID3_FRAMES.getbuffer().nbytes

# 크기를 28비트로 인코딩 합니다.
encode_size = encode_size(frame_size)

# 모든 프레임과 헤더를 작성하여 최종 ID3v2.3 태그를 작성합니다.
ID3_TAG = BytesIO()
ID3_TAG.write(b"ID3")
ID3_TAG.write(struct.pack("h", 3))
ID3_TAG.write(b"\x00")
ID3_TAG.write(struct.pack(">I", int(encode_size, 16)))
ID3_TAG.write(ID3_FRAMES.read())
ID3_TAG.seek(0)

# 최초 ID3v2 태그가 제거되고 남은 파일을 ID3v2 태그와 함께 파일로 기록합니다.
with open("delete_idtag.mp3", "wb") as out:
    out.write(ID3_TAG.read())
    out.write(delete_tag_mp3.read())

![img](images/12.jpg)

위의 코드는 가사, 커버이미지, 트랙, 디스크 번호, 커멘트 등을 추가하여 작성된 코드입니다. 물론 이해를 돕기 위해 최대한 직설적인 방식으로 코딩했습니다. 이렇게 위의 코드를 실행하여 작성된 delete_idtag.mp3 파일을 mp3tag 프로그램으로 열어보면 위의 이미지에서 처럼 정상적으로 태그가 작성된것을 확인 할 수 있습니다.

![img](images/13.jpg)

윈도우10 기본 탐색기의 파일 속성에서도 정상적으로 인식하는것을 확인했습니다. 물론 탐색기의 파일속성 기능으로는 커버이미지나 가사는 출력되지 않습니다.

![img](images/14.jpg)

다음 팟 플레이어로 해당 파일을 재생하면 커버이미지, 가사를 인식하는것을 확인할 수 있습니다. 물론 비동기 가사 형태로 저장된 데이터기 때문에 노래방처럼 가사를 시간별로 인식하진 않고 해당 기능은 공식사이트의 <a href="https://id3.org/id3v2.3.0#sec4.10">4.10 섹션</a> "SYLT" 항목을 확인해보시기 바랍니다.